In [1]:
import pandas as pd
import numpy as np

In [571]:
class Time:
    def __init__(self, name, c0, c1, h0, h1, penalty, demand, setupfee, setupfee2, isProduct0, cur_period_index):
        self.index = int(name)
        self.production_cost_0 = float(c0)
        self.production_cost_1 = float(c1)
        self.holding_cost_0 = float(h0)
        self.holding_cost_1 = float(h1)
        self.penalty = float(penalty)
        self.demand = float(demand)
        self.setupfee = float(setupfee)
        self.setupfee2 = float(setupfee2)
        self.Product0 = bool(isProduct0)
        self.CurPeriodIndex = int(cur_period_index)
        self.cum_demand_for_production = {}
        self.cum_demand_for_holding = {}
        self.used = 1 # surgical mask used

    def __str__(self):
        return "time Index: " + str(self.index) + ", c0 = " + str(self.production_cost_0) + ", c1 = " + str(self.production_cost_1) + ", h0 = " + str(self.holding_cost_0) + ", h1 = " + str(self.holding_cost_1) + ", penalty = " + str(self.penalty) + ", demand = " + str(self.demand) + ", setupfee_n95 = " + str(self.setupfee) + ", setupfee_surgical = " + str(self.setupfee2) +  ", can product 0 = " + str(self.Product0) + ", cur Time Period = " + str(self.CurPeriodIndex)

    def printCumDemandForProduct(self):
        print("Total demand for production at Time ", self.index, " is ", self.cum_demand_for_production)

    def printCumDemandForHolding(self):
        print("Total demand for holding at Time ", self.index, " is ", self.cum_demand_for_holding)

    def printWhichSatisfyDemand(self):
        print("The production satisfies demand at Time ", self.index, " is ", self.used)

class TIME:
    def __init__(self, index, begin, end):
        self.index = index
        self.begin_time = begin
        self.end_time = end
        self.best_policy = {}   # key: TIME index, value: policy1 = -1, policy2 = -2, policy3 = positive number
        self.best_cost = {}   # key: TIME index, value: cost
        self.best = tuple()  # (cover to when, policy num, policy cost)

    def __str__(self):
        return "TIME index: " + str(self.index) + ", begin_time = " + str(self.begin_time) + ", end_time = " + str(self.end_time)

    def printPolicy(self):
        key_list = list(self.best_policy.keys())
        for i in key_list:
            if self.best_policy[i] == -1:
                print("Best policy for TIME ", self.index, "to TIME ", i, " is: only order N95, and the cost is ", self.best_cost[i])
            elif self.best_policy[i] == -2:
                print("Best policy for TIME ", self.index, "to TIME ", i, " is: only order surgical mask, and the cost is ", self.best_cost[i])
            else:
                print("Best policy for TIME ", self.index, "to TIME ", i, " is: order N95 to cover time ", self.best_policy[i], ", and then order surgical mask, and the total cost is ", self.best_cost[i])

class PPE:
    def __init__(self, file_path, setupfee):
        self.TimeSeries = []
        self.TIMESeries = []
        self.product0_index = {}
        self.setUpFee = setupfee
        self.N95range = []

        count = 0
        cur_T = -1
        with open(file_path, "r") as file:
            for line in file:
                if count == 0:
                    self.TotalTime = int(line.strip())
                elif count == 1:
                    string_list = line.strip().split(" ")
                    self.Period = [int(x) for x in string_list]
                else:
                    tmp_c0, tmp_c1, tmp_h0, tmp_h1, tmp_p, tmp_d, tmp_setup, tmp_setup2, tmp_product0 = line.strip().split(" ")

                    if tmp_product0 == "1":
                        cur_T += 1
                        self.product0_index[cur_T] = count - 2
                        tmp_TIME = TIME(cur_T, count - 2, count - 3 + self.Period[cur_T])
                        self.TIMESeries.append(tmp_TIME)   # add big time period to TIME Series

                    tmp_t = Time(count - 2, tmp_c0, tmp_c1, tmp_h0, tmp_h1, tmp_p, tmp_d, tmp_setup, tmp_setup2, tmp_product0, cur_T)
                    last_t = self.TimeSeries[-1] if len(self.TimeSeries) > 0 else None   # record the previous one

                    self.TimeSeries.append(tmp_t)   # add small time period to Time Series

                    self.calculate_cumProductAndHoldingQuantity(tmp_t, last_t)   # update the cum demand for production and holding

                    # tmp_t.printCumDemandForProduct()
                    # tmp_t.printCumDemandForHolding()
                count += 1

        self.cal_best_policy()

    def backward(self, representInTIME):
        tmp_index = len(self.TIMESeries) - 1
        tmp_best = {}
        temp_v = {}
        temp_p = {}

        for i in range(tmp_index, -1, -1):
            tmp_min = 100000000
            tmp_only = 0
            tmp_j = tmp_index
            for j in range(i, tmp_index + 1):
                if j < tmp_index:
                    # print("old, i = ", i, ", j = ", j, ", cost = ", temp_v[j+1] + self.TIMESeries[i].best_cost[j], "temp_v[j+1] = ", temp_v[j+1], ", policy = ", self.TIMESeries[i].best_policy[j])
                    if temp_v[j+1] + self.TIMESeries[i].best_cost[j] < tmp_min:
                        tmp_min = temp_v[j+1] + self.TIMESeries[i].best_cost[j]
                        tmp_only = self.TIMESeries[i].best_cost[j]
                        temp_p[i] = self.TIMESeries[i].best_policy[j]
                        tmp_j = j
                else:
                    # print("new, i = ", i, ", j = ", j, ", cost = ", self.TIMESeries[i].best_cost[j], ", policy = ", self.TIMESeries[i].best_policy[j])
                    if self.TIMESeries[i].best_cost[j] < tmp_min:
                        tmp_min = self.TIMESeries[i].best_cost[j]
                        tmp_only = self.TIMESeries[i].best_cost[j]
                        temp_p[i] = self.TIMESeries[i].best_policy[j]
                        tmp_j = j
            temp_v[i] = tmp_min
            # print("temp_v[", i, "] = ", temp_v[i])
            self.TIMESeries[i].best = (tmp_j, temp_p[i], tmp_only, temp_v[i])   # (cover to, policy, current cost, total cost)
            # print("Best policy for TIME ", self.TIMESeries[i].index, "to TIME ", tmp_j, " is: ", temp_p[i], ", and the cost is ", tmp_min)

        cur = 0
        while cur <= tmp_index:
            tmp_best[cur] = self.TIMESeries[cur].best

            if self.TIMESeries[cur].best[1] == -1:
                for h in range(cur, self.TIMESeries[cur].best[0] + 1):
                    for j in range(self.TIMESeries[h].begin_time, self.TIMESeries[h].end_time + 1):
                        self.TimeSeries[j].used = 0
                temp = (cur, self.TIMESeries[cur].begin_time, self.TIMESeries[cur].best[0], self.TIMESeries[self.TIMESeries[cur].best[0]].end_time)
                self.N95range.append(temp)   # (T, t, T, t)

            if self.TIMESeries[cur].best[1] >= 0:
                for h in range(cur, self.TIMESeries[cur].best[0] + 1):
                    for j in range(self.TIMESeries[h].begin_time, self.TIMESeries[h].end_time + 1):
                        self.TimeSeries[j].used = 0
                        if j == self.TIMESeries[cur].best[1]:
                            break
                temp = (cur, self.TIMESeries[cur].begin_time, self.TIMESeries[cur].best[0], self.TIMESeries[cur].best[1])
                self.N95range.append(temp)

            if cur == tmp_index:
                break
            else:
                cur = self.TIMESeries[cur].best[0] + 1

        key_list = list(tmp_best.keys())
        print("The total cost is ", tmp_best[0][3], ".")
        for i in key_list:
            if representInTIME:
                if (tmp_best[i][1] == -1):
                    print("Best policy for TIME ", i, "to TIME ", tmp_best[i][0], " is: only order N95, and the cost is ", tmp_best[i][2])
                elif (tmp_best[i][1] == -2):
                    print("Best policy for TIME ", i, "to TIME ", tmp_best[i][0], " is: only order surgical mask, and the cost is ", tmp_best[i][2])
                else:
                    print("Best policy for TIME ", i, "to TIME ", tmp_best[i][0], " is: order N95 to cover time ", tmp_best[i][1], ", and then order surgical mask, and the total cost is ", tmp_best[i][2])
            else:
                if (tmp_best[i][1] == -1):
                    print("Best policy for time ", self.TIMESeries[i].begin_time, "to time ", self.TIMESeries[tmp_best[i][0]].end_time, " is: only order N95, and the cost is ", tmp_best[i][2])
                elif (tmp_best[i][1] == -2):
                    print("Best policy for time ", self.TIMESeries[i].begin_time, "to time ", self.TIMESeries[tmp_best[i][0]].end_time, " is: only order surgical mask, and the cost is ", tmp_best[i][2])
                else:
                    print("Best policy for time ", self.TIMESeries[i].begin_time, "to time ", self.TIMESeries[tmp_best[i][0]].end_time, " is: order N95 to cover time ", tmp_best[i][1], ", and then order surgical mask, and the total cost is ", tmp_best[i][2])

    @staticmethod
    def calculate_cumProductAndHoldingQuantity(tmp_t, last_t):
        if tmp_t.index == 0:
            tmp_t.cum_demand_for_production[0] = tmp_t.demand
            tmp_t.cum_demand_for_holding[0] = 0
        else:
            for i in range(tmp_t.index):
                tmp_t.cum_demand_for_production[i] = last_t.cum_demand_for_production[i] + tmp_t.demand
                tmp_t.cum_demand_for_holding[i] = last_t.cum_demand_for_holding[i] + tmp_t.demand
            tmp_t.cum_demand_for_production[tmp_t.index] = tmp_t.demand
            tmp_t.cum_demand_for_holding[tmp_t.index] = 0

    def __len__(self):
        return self.TotalTime

    def __str__(self):
        print("TotalTime: " + str(self.TotalTime) + ", Period: " + str(self.Period))
        print("product0_index: ", self.product0_index)
        for t in self.TimeSeries:
            print(t)
        return ""

    # policy1: only order p0
    # policy2: only order p1
    # policy3: order p0 and p1 mix, need to find the best mix
    def cal_best_policy(self):
        for T in self.TIMESeries:
            for i in range(T.index, self.TIMESeries[-1].index + 1):
                T1 = self.TIMESeries[i]
                # print("In cal_best_policy, T = ", T.index, ", T1 = ", T1.index)
                policy1 = self.cal_policy1(T, T1, self.setUpFee)
                policy2 = self.cal_policy2(T, T1)
                policy3, best_time_policy3 = self.cal_policy3(T, T1, self.setUpFee)
                # print("Policy 1: ", policy1)
                # print("Policy 2: ", policy2)
                # print("Policy 3: ", policy3, " the best time is: ", best_time_policy3)
                best_policy  = min(policy1, policy2, policy3)   # minimal cost

                if best_policy == policy1:
                    T.best_policy[T1.index] = -1
                    T.best_cost[T1.index] = best_policy
                elif best_policy == policy2:
                    T.best_policy[T1.index] = -2
                    T.best_cost[T1.index] = best_policy
                else:
                    T.best_policy[T1.index] = best_time_policy3
                    T.best_cost[T1.index] = best_policy
            # T.printPolicy()   # check best policy

    def cal_policy1(self, tmp_T, tmp_T1, SetUpFee):
        produce_cost = 0
        hold_cost = 0
        for i in range(tmp_T.index, tmp_T1.index + 1):
            temp = self.TIMESeries[i]

            if SetUpFee:   # if setup fee is considered
                produce_cost += self.TimeSeries[temp.begin_time].setupfee   # setup fee

            produce_cost += self.TimeSeries[temp.begin_time].production_cost_0 * self.TimeSeries[temp.end_time].cum_demand_for_production[temp.begin_time]    # c_start * d(end, start)

            for j in range(temp.begin_time, temp.end_time + 1):
                hold_cost += self.TimeSeries[j].holding_cost_0 * self.TimeSeries[temp.end_time].cum_demand_for_holding[j]   # h_j * d(end, j)
        return produce_cost + hold_cost

    def cal_policy2(self, tmp_T, tmp_T1):
        produce_and_penalty_cost = 0
        for i in range(tmp_T.index, tmp_T1.index + 1):
            temp = self.TIMESeries[i]
            for i in range(temp.begin_time, temp.end_time + 1):
                produce_and_penalty_cost += (self.TimeSeries[i].production_cost_1 + self.TimeSeries[i].penalty) * self.TimeSeries[i].cum_demand_for_production[i]   # (c1 + p) * d(i, i)

        return produce_and_penalty_cost

    def cal_policy3(self, tmp_T, tmp_T1, SetUpFee):
        best_time = -1
        best_cost = 10000000000
        new_TIME_criteria = []   # [4, 8]
        for i in range(tmp_T.index, tmp_T1.index+1):
            new_TIME_criteria.append(self.product0_index[i])

        for i in range(tmp_T.begin_time, tmp_T1.end_time+1):
            temp_cost_production0 = self.TimeSeries[tmp_T.begin_time].setupfee
            temp_cost_holding0 = 0
            temp_production_and_penalty_cost = 0
            temp_total_cost = 0
            # temp = self.TimeSeries[i]   # temp is small time period

            # print(i, " ", new_TIME_criteria)   # check setup fee
            # if SetUpFee:   # if setup fee is considered
            #     for tempt in new_TIME_criteria:
            #         if tempt <= temp.index:
            #             temp_cost_production0 += self.TimeSeries[tempt].setupfee   # setup fee
            # print(i, " ", temp_cost_production0)   # check setup fee

            temp_cost_production0 += self.TimeSeries[tmp_T.begin_time].production_cost_0 * self.TimeSeries[i].cum_demand_for_production[tmp_T.begin_time]   # c_T's begin time * d(i, T's begin time)

            for j in range(tmp_T.begin_time, i + 1):
                temp_cost_holding0 += self.TimeSeries[j].holding_cost_0 * self.TimeSeries[i].cum_demand_for_holding[j]   # h_j * d(i, j)

            for j in range(i+1, tmp_T1.end_time + 1):
                temp_production_and_penalty_cost += (self.TimeSeries[j].production_cost_1 + self.TimeSeries[j].penalty) * self.TimeSeries[j].cum_demand_for_production[j]   # (c1 + p) * d(j, j)
            # print(temp_production_and_penalty_cost)

            temp_total_cost = temp_cost_production0 + temp_cost_holding0 + temp_production_and_penalty_cost

            if temp_total_cost <= best_cost:
                best_cost = temp_total_cost
                best_time = i

        return best_cost, best_time

    def printN95Range(self):
        print("N95 range: ", self.N95range)

    def checkSatisfy(self):
        for i in range(len(self.TimeSeries)):
            self.TimeSeries[i].printWhichSatisfyDemand()

    def extend_N95_policy(self, Time_st, Time_add, TIME_st, TIME_add):  # Time_st: start time period, Time_add: add time period
        temp = 0
        if TIME_st != TIME_add:
            temp = self.TimeSeries[self.TIMESeries[TIME_add].begin_time].production_cost_0 * self.TimeSeries[Time_add].cum_demand_for_production[self.TIMESeries[TIME_add].begin_time] + self.TimeSeries[self.TIMESeries[TIME_add].begin_time].setupfee   # production cost at the last time period
            # print(self.TimeSeries[self.TIMESeries[TIME_add].begin_time].production_cost_0 * self.TimeSeries[Time_add].cum_demand_for_production[self.TIMESeries[TIME_add].begin_time])
            # print(self.TimeSeries[self.TIMESeries[TIME_add].begin_time].setupfee)
            for i in range(self.TIMESeries[TIME_add].begin_time, Time_add):   # holding cost
                temp += self.TimeSeries[i].holding_cost_0 * self.TimeSeries[Time_add].cum_demand_for_holding[i]
            # print(self.TimeSeries[Time_add].cum_demand_for_production[self.TIMESeries[TIME_add].begin_time], " ", temp)

            for i in range(TIME_st, TIME_add):   # productiomn at the begining of the time period
                temp += self.TimeSeries[self.TIMESeries[i].begin_time].production_cost_0 * self.TimeSeries[self.TIMESeries[i].end_time].cum_demand_for_production[self.TIMESeries[i].begin_time] + self.TimeSeries[self.TIMESeries[i].begin_time].setupfee   # production cost at the last time period
                # print(self.TimeSeries[self.TIMESeries[i].begin_time].production_cost_0 * self.TimeSeries[self.TIMESeries[i].end_time].cum_demand_for_production[self.TIMESeries[i].begin_time])
                # print(self.TimeSeries[self.TIMESeries[i].begin_time].setupfee)
                for j in range(self.TIMESeries[i].begin_time, self.TIMESeries[i].end_time + 1):   # holding cost
                    temp += self.TimeSeries[j].holding_cost_0 * self.TimeSeries[self.TIMESeries[i].end_time].cum_demand_for_holding[j]
                    # print(self.TimeSeries[self.TIMESeries[i].end_time].cum_demand_for_holding[j])
        else:   # in the same T
            temp = self.TimeSeries[Time_st].production_cost_0 * self.TimeSeries[Time_add].cum_demand_for_production[Time_st] + self.TimeSeries[Time_st].setupfee
            for i in range(Time_st, Time_add):
                temp += self.TimeSeries[i].holding_cost_0 * self.TimeSeries[Time_add].cum_demand_for_holding[i]

        return temp   # return the cost of extend N95 policy

    def find_surgical_policy(self, Time_st, Time_ed):
        # print("*****", Time_st, "*****", Time_ed)
        res = {}
        for i in range(Time_st, Time_ed + 1):
            res[i] = {}
            for j in range(i, Time_ed + 1):
                # print("***", i, "***", j)
                if i == j:
                    tmp = (self.TimeSeries[j].production_cost_1 + self.TimeSeries[j].penalty) * self.TimeSeries[j].cum_demand_for_production[i] + self.TimeSeries[j].setupfee2
                    res[i][j] = tmp
                else:
                    tmp = (self.TimeSeries[i].production_cost_1 + self.TimeSeries[j].penalty) * self.TimeSeries[j].cum_demand_for_production[i] + self.TimeSeries[i].setupfee2
                    for k in range(i, j):
                        tmp += self.TimeSeries[k].holding_cost_1 * self.TimeSeries[j].cum_demand_for_holding[k]
                    res[i][j] = tmp

        best_v = {}
        best_j = {}
        best = {}
        for i in range(Time_ed, Time_st-1, -1):
            # print(i)
            tmp_min = 100000000
            tmp_only = 0
            for j in range(i, Time_ed + 1):
                if j < Time_ed:
                    # print("old, i = ", i, ", cover to = ", j, ", cost = ", best_v[j+1] + res[i][j], "temp_v[j+1] = ", best_v[j+1])
                    if best_v[j+1] + res[i][j] <= tmp_min:
                        tmp_min = best_v[j+1] + res[i][j]
                        tmp_only = res[i][j]
                        best_j[i] = j
                else:
                    # print("new, i = ", i, ", cover to = ", j, ", cost = ", res[i][j])
                    if res[i][j] <= tmp_min:
                        tmp_min = res[i][j]
                        tmp_only = res[i][j]
                        best_j[i] = j
            best_v[i] = tmp_min
            # print("best_v[", i, "] = ", best_v[i])
            best[i] = (best_j[i], tmp_only, best_v[i])   # (cover to, cost)
            # print("Best policy is from time ", i, "to time ", best_j[i], " , and the cost is ", best_v[i])

        cur = Time_st
        total_cost_surgical = best[cur][2]
        res_best = []
        # print("\nThe total cost for surgical mask from time", Time_st, "to time", Time_ed, "is ", total_cost_surgical, ".")
        while cur <= Time_ed:
            res_best.append([cur, best[cur][0], best[cur][1]])
            # print("Best policy is from time ", cur, "to time ", best[cur][0], ", and the cost is ", best[cur][1])
            if cur == Time_ed:
                break
            else:
                cur = best[cur][0] + 1

        return total_cost_surgical, res_best

    def backward2(self):
        res = {}
        surgical_res_policy = {}
        for i in range(len(self.N95range)):
            cur_total = 100000000
            record_j = -1
            if i < len(self.N95range) - 1:
                cur_ed = self.N95range[i][3]   # the last one in the current range
                next_start = self.N95range[i+1][1]   # the first one in the next range
                if cur_ed == next_start - 1:
                    continue
                # print("&&&&&&", cur_ed, " ", next_start)
                for j in range(cur_ed, next_start):
                    cur_n95_cost = self.extend_N95_policy(self.N95range[i][1], j, self.N95range[i][0], self.TimeSeries[j].CurPeriodIndex)
                    # print("*******", j)
                    # print("start: ", self.N95range[i][1], "j: ", j, "cur_n95_cost = ", cur_n95_cost)
                    if j + 1 <= next_start:
                        cur_surgical_cost, cur_surgical_policy = self.find_surgical_policy(j, next_start-1)
                        tmp_cur_total = cur_n95_cost + cur_surgical_cost
                        if tmp_cur_total <= cur_total:   # (T, t, T, t)
                            record_j = j
                            cur_total = tmp_cur_total
                            surgical_res_policy[i] = cur_surgical_policy
                    else:
                        tmp_cur_total = cur_n95_cost
                        if tmp_cur_total <= cur_total:   # (T, t, T, t)
                            record_j = j
                            cur_total = tmp_cur_total
                            surgical_res_policy = {}

                    # print("totalcost = ", cur_total)

            else:
                cur_ed = self.N95range[i][3]   # the last one in the current range
                # print(self.N95range[i])
                # print("cur_ed:", cur_ed)
                next_start = self.TIMESeries[-1].end_time   # the first one in the next range
                # print("cur_ed: ", cur_ed, "next: ", next_start)

                for j in range(cur_ed, next_start + 1):
                    # print("$$$$$$$$", j)
                    cur_n95_cost = self.extend_N95_policy(self.N95range[i][1], j,  self.N95range[i][0], self.TimeSeries[j].CurPeriodIndex)
                    # print(self.N95range[i][1], j,  self.N95range[i][0], self.TimeSeries[j].CurPeriodIndex)
                    # print("start: ", self.N95range[i][1], "j: ", j, "cur_n95_cost = ", cur_n95_cost)
                    if j + 1 <= next_start:
                        # print("j = ", j, "next_start = ", next_start)
                        cur_surgical_cost, cur_surgical_policy = self.find_surgical_policy(j + 1, next_start)
                        tmp_cur_total = cur_n95_cost + cur_surgical_cost
                        if tmp_cur_total <= cur_total:   # (T, t, T, t)
                            record_j = j
                            cur_total = tmp_cur_total
                            surgical_res_policy[i] = cur_surgical_policy
                    else:
                        tmp_cur_total = cur_n95_cost
                        if tmp_cur_total <= cur_total:   # (T, t, T, t)
                            record_j = j
                            cur_total = tmp_cur_total
                            surgical_res_policy = {}
                    # print("totalcost = ", cur_total)


            temp = (self.N95range[i][0], self.N95range[i][1], self.TimeSeries[record_j].CurPeriodIndex, record_j, cur_total)
            # print(temp)
            res[i] = temp
        return res, surgical_res_policy
                        # print("update: ", self.N95range[i][0], self.N95range[i][3], self.N95range[i][4])




In [577]:
ppe = PPE("/Users/pikay/Documents/PPE/data/1.txt", True)   # True: setup fee, False: no setup fee
ppe.backward(False)  # True: TIME, False: time
res, policy = ppe.backward2()
# ppe.checkSatisfy()
# tmp_cost, tmp_p = ppe.find_surgical_policy(1, 3)

The total cost is  3467.0 .
Best policy for time  0 to time  3  is: order N95 to cover time  0 , and then order surgical mask, and the total cost is  1179.0
Best policy for time  4 to time  7  is: order N95 to cover time  4 , and then order surgical mask, and the total cost is  1085.0
Best policy for time  8 to time  11  is: only order surgical mask, and the cost is  1203.0


In [578]:
res

{0: (0, 0, 0, 1, 1643.0), 1: (1, 4, 1, 5, 2498.0)}

In [579]:
policy

{0: [[1, 2, 688.0], [3, 3, 390.0]],
 1: [[6, 6, 282.0],
  [7, 7, 324.0],
  [8, 8, 330.0],
  [9, 9, 310.0],
  [10, 10, 330.0],
  [11, 11, 353.0]]}

In [580]:
ppe.printN95Range()

N95 range:  [(0, 0, 0, 0), (1, 4, 1, 4)]
